In [ ]:
from env import generate_url

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import LabelEncoder 

In [ ]:
url = generate_url()

In [ ]:
list(pd.read_sql('SHOW DATABASES;',url)['Database'])

In [ ]:
dburl = generate_url('mall_customers')

In [ ]:
pd.read_sql("SHOW TABLES", dburl)

## Grabs DF

In [ ]:
df = pd.read_sql("SELECT * FROM customers", dburl)

In [ ]:
df.describe()

In [ ]:
sns.distplot(df.annual_income)

In [ ]:
q3 = df.annual_income.quantile(q=.75)

In [ ]:
q1 = df.annual_income.quantile(q=.25)

In [ ]:
iqr = q3 - q1 

In [ ]:
iqr_scaled = iqr * 1.5

In [ ]:
iqr_scaled - q1

In [ ]:
outlier_floor = iqr_scaled - q1

In [ ]:
outlier_ciel = iqr_scaled + q3

In [ ]:
def detect_outliers(df, col = 'annual_income'):
    q3 = df[col].quantile(q=.75)
    q1 = df[col].quantile(q=.25)
    iqr = q3 - q1
    iqr_scaled = iqr*1.5
    outlier_floor = iqr_scaled - q1
    outlier_ciel = iqr_scaled + q3
    outlier_df = pd.concat([df[df[col] > outlier_ciel],df[df[col] < outlier_floor]])
    return outlier_df

In [ ]:
df = pd.concat([df[df['annual_income'] > outlier_ciel], df[df['annual_income'] < outlier_floor]])

In [ ]:
detect_outliers(df)

In [ ]:
df.info()

In [ ]:
def encode_gender(df):
    encoder = LabelEncoder()
    encoder.fit(df.gender)
    return df.assign(gender = encoder.transform(df.gender))

In [ ]:
encode_gender(df).info()

In [ ]:
oldf = detect_outliers(df)
oldf

In [ ]:
for i in oldf.index:
    df.drop(i)

In [ ]:
def drop_outliers(df):
    outliers = detect_outliers(df)
    dfc = df.copy()
    for i in outliers.index:
        dfc.drop(i,inplace=True)
    return dfc 

In [ ]:
drop_outliers(df)

In [ ]:
df

In [ ]:
import prepare_mall

In [ ]:
df.info()

In [ ]:
prepare_mall.drop_outliers(df)

In [ ]:
def drop_outliers(df):
    outliers = detect_outliers(df)
    dfc = df.copy()
    for i in outliers.index:
        dfc.drop(i,inplace=True)
    return dfc

In [ ]:
drop_outliers(df)

In [ ]:
df

##  ZILLOW DATA ENGINEERING

In [ ]:
dburl = generate_url('zillow')
dfz = pd.read_sql('SELECT * FROM properties_2016 JOIN predictions_2016 USING (parcelid)',dburl)
tables = pd.read_sql('SHOW TABLES',dburl)
tables.drop(4,inplace=True)
tables.drop(5,inplace=True)
tables.drop(6,inplace=True)
tables.drop(7,inplace=True)
tables.drop(11,inplace=True)

type_dfs = []
for i in list(tables.Tables_in_zillow):
    type_dfs.append(pd.read_sql('SELECT * FROM '+i,dburl))
    
dfz2 = dfz
for i in type_dfs:
    dfz2 = pd.merge(dfz2,i,how="left")
    


In [ ]:
dfz.shape

In [ ]:
tables = pd.read_sql('SHOW TABLES',dburl)

In [ ]:

tables

In [ ]:
tables.drop(4,inplace=True)
tables.drop(5,inplace=True)
tables.drop(6,inplace=True)
tables.drop(7,inplace=True)
tables.drop(11,inplace=True)


In [ ]:
list(tables.Tables_in_zillow)

In [ ]:
type_dfs = []
for i in list(tables.Tables_in_zillow):
    type_dfs.append(pd.read_sql('SELECT * FROM '+i,dburl))

In [ ]:
list(type_dfs[0])[0]

In [ ]:
dfz2 = dfz
for i in type_dfs:
    dfz2 = pd.merge(dfz2,i,how="left")

In [ ]:
dfz2

In [1]:
import acquire_zillow

In [ ]:
dfz = acquire_zillow.get_2016_properties()

In [ ]:
import pandas as pd 
dfz.shape

In [ ]:
dfz2 = acquire_zillow.get_2017_properties()

In [ ]:
dfz2.shape

In [ ]:
def all_zillow():
    
    return pd.concat([dfz,dfz2])
    

In [ ]:
dfz3 = all_zillow()

In [ ]:
dfz.head()

In [ ]:
dfz2.head()

In [ ]:
dfz3.head()

In [ ]:
dfz3.drop('id',axis=1).shape

In [ ]:
dfz

In [3]:
dfz = acquire_zillow.get_1617()

In [7]:
(dfz[~dfz.longitude.isnull() | ~dfz.latitude.isnull()]).shape

(167854, 67)

In [2]:
acquire_zillow.zillow_to_csv('test.csv')

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('test.csv')

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167854 entries, 0 to 167853
Data columns (total 68 columns):
Unnamed: 0                      167854 non-null int64
parcelid                        167854 non-null int64
airconditioningtypeid           53788 non-null float64
architecturalstyletypeid        468 non-null float64
basementsqft                    93 non-null float64
bathroomcnt                     167854 non-null float64
bedroomcnt                      167854 non-null float64
buildingclasstypeid             31 non-null float64
buildingqualitytypeid           107173 non-null float64
calculatedbathnbr               166056 non-null float64
decktypeid                      1272 non-null float64
finishedfloor1squarefeet        12893 non-null float64
calculatedfinishedsquarefeet    166992 non-null float64
finishedsquarefeet12            159519 non-null float64
finishedsquarefeet13            75 non-null float64
finishedsquarefeet15            6591 non-null float64
finishedsquarefeet

In [41]:
i = df[df['bedroomcnt' ]==0]
j = i[i['bathroomcnt'] ==0]

In [38]:
df.drop(j.index).shape[0]

166130

In [42]:
j.shape

(1724, 68)

In [36]:
df.shape[0] - j.shape[0]

166130

In [47]:
df.unitcnt.value_counts(dropna = False)

 1.0      101916
NaN        58798
 2.0        4385
 4.0        1597
 3.0        1148
 6.0           2
 45.0          1
 11.0          1
 237.0         1
 42.0          1
 5.0           1
 9.0           1
 143.0         1
 70.0          1
Name: unitcnt, dtype: int64

In [61]:
type(df.unitcnt[1])

numpy.float64

In [69]:
df.unitcnt = df.unitcnt.replace(np.nan,1)

In [74]:
df.unitcnt.value_counts(dropna = 0)

1.0      160714
2.0        4385
4.0        1597
3.0        1148
6.0           2
45.0          1
11.0          1
237.0         1
42.0          1
5.0           1
9.0           1
143.0         1
70.0          1
Name: unitcnt, dtype: int64

In [76]:
list(df.columns)

['Unnamed: 0',
 'parcelid',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'heatingorsystemtypeid',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'yearbuilt',
 'numberofst

In [84]:
df[['bathroomcnt',
 'bedroomcnt']]
df.bedroomcnt.value_counts(dropna=0)
#df.bathroomcnt.value_counts(dropna=0)

3.0     65883
2.0     41580
4.0     37830
5.0      9627
1.0      7288
0.0      2258
6.0      2120
8.0       527
7.0       442
9.0       161
10.0       66
12.0       38
11.0       21
16.0        5
14.0        4
13.0        3
15.0        1
Name: bedroomcnt, dtype: int64

In [82]:
def percent_zeros(df):
    cols = list(df.columns)
    zeros = {}
    for col in cols:
        zeros[col] = 'Zeros: {:.2f}%'.format((df[col] == 0).sum()/df.shape[0] * 100)
        
    return zeros

def percent_NaNs(df):
    cols = list(df.columns)
    Nans = {}
    for col in cols:
        Nans[col] = 'Missing {:0.2f}%'.format(df[col].isnull().sum()/df.shape[0]*100)
        
    return Nans

In [83]:
percent_NaNs(df)

{'Unnamed: 0': 'Missing 0.00%',
 'parcelid': 'Missing 0.00%',
 'airconditioningtypeid': 'Missing 67.96%',
 'architecturalstyletypeid': 'Missing 99.72%',
 'basementsqft': 'Missing 99.94%',
 'bathroomcnt': 'Missing 0.00%',
 'bedroomcnt': 'Missing 0.00%',
 'buildingclasstypeid': 'Missing 99.98%',
 'buildingqualitytypeid': 'Missing 36.15%',
 'calculatedbathnbr': 'Missing 1.07%',
 'decktypeid': 'Missing 99.24%',
 'finishedfloor1squarefeet': 'Missing 92.32%',
 'calculatedfinishedsquarefeet': 'Missing 0.51%',
 'finishedsquarefeet12': 'Missing 4.97%',
 'finishedsquarefeet13': 'Missing 99.96%',
 'finishedsquarefeet15': 'Missing 96.07%',
 'finishedsquarefeet50': 'Missing 92.32%',
 'finishedsquarefeet6': 'Missing 99.52%',
 'fips': 'Missing 0.00%',
 'fireplacecnt': 'Missing 89.34%',
 'fullbathcnt': 'Missing 1.07%',
 'garagecarcnt': 'Missing 66.96%',
 'garagetotalsqft': 'Missing 66.96%',
 'hashottuborspa': 'Missing 97.67%',
 'heatingorsystemtypeid': 'Missing 37.06%',
 'latitude': 'Missing 0.00%',
 

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167854 entries, 0 to 167853
Data columns (total 68 columns):
Unnamed: 0                      167854 non-null int64
parcelid                        167854 non-null int64
airconditioningtypeid           53788 non-null float64
architecturalstyletypeid        468 non-null float64
basementsqft                    93 non-null float64
bathroomcnt                     167854 non-null float64
bedroomcnt                      167854 non-null float64
buildingclasstypeid             31 non-null float64
buildingqualitytypeid           107173 non-null float64
calculatedbathnbr               166056 non-null float64
decktypeid                      1272 non-null float64
finishedfloor1squarefeet        12893 non-null float64
calculatedfinishedsquarefeet    166992 non-null float64
finishedsquarefeet12            159519 non-null float64
finishedsquarefeet13            75 non-null float64
finishedsquarefeet15            6591 non-null float64
finishedsquarefeet

In [90]:
df.drop(df[df.unitcnt>1].index)

,Unnamed: 0,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,0,11016594,1.0,NaN,NaN,2.0,3.0,NaN,4.0,2.0,...,6.037107e+13,0.027600,2016-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
1,1,14366692,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,NaN,-0.168400,2016-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,2,12098116,1.0,NaN,NaN,3.0,2.0,NaN,4.0,3.0,...,6.037464e+13,-0.004000,2016-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
3,3,12643413,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,...,6.037296e+13,0.021800,2016-01-02,Central,NaN,NaN,Central,Condominium,NaN,NaN
4,4,14432541,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,...,6.059042e+13,-0.005000,2016-01-02,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
5,5,11509835,1.0,NaN,NaN,4.0,4.0,NaN,1.0,4.0,...,6.037621e+13,-0.270500,2016-01-02,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
6,6,12286022,NaN,NaN,NaN,1.0,2.0,NaN,7.0,1.0,...,6.037542e+13,0.044000,2016-01-02,NaN,NaN,NaN,Floor/Wall,Single Family Residential,NaN,NaN
7,7,17177301,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,...,6.111003e+13,0.163800,2016-01-02,NaN,NaN,NaN,NaN,Condominium,NaN,NaN
8,8,14739064,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.059042e+13,-0.003000,2016-01-02,NaN,NaN,NaN,NaN,Condominium,NaN,NaN
9,9,14677559,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,...,6.059063e+13,0.084300,2016-01-03,NaN,NaN,NaN,NaN,Condominium,NaN,NaN
